In [1]:
from mednlpix.data.loader import get_medical_dataset
from mednlpix.data.splitter import split_medical_dataset

In [2]:
print("\n" + "="*60)
print("Load Medical Dataset")
print("="*60 + "\n")

data, dataset_path = get_medical_dataset(return_path=True)

print(f"Loaded dataset: {dataset_path.name}, shape = {data.shape}")
display(data.head())

2025-10-07 14:01:36,574 - mednlpix.data.loader - INFO - Searching for dataset in: /Users/surelmanda/Data-Science-Projects/MedNLPix/data/raw
2025-10-07 14:01:36,576 - mednlpix.data.loader - INFO - Dataset found: medical-abstracts-dataset.csv
2025-10-07 14:01:36,577 - mednlpix.data.loader - INFO - Loading dataset from: /Users/surelmanda/Data-Science-Projects/MedNLPix/data/raw/medical-abstracts-dataset.csv
2025-10-07 14:01:36,609 - mednlpix.data.loader - INFO - Dataset loaded successfully — the input data has 2286 rows and 2 columns.
2025-10-07 14:01:36,610 - mednlpix.data.loader - INFO - Dataset successfully retrieved and loaded.



Load Medical Dataset

Loaded dataset: medical-abstracts-dataset.csv, shape = (2286, 2)


,condition_label,medical_abstract
0,cardiovascular diseases,Coronary artery bypass grafting in a patient w...
1,cardiovascular diseases,Conservative management of aortic lacerations ...
2,cardiovascular diseases,Sudden death from acute cocaine intoxication i...
3,cardiovascular diseases,Catheterization of coronary artery bypass graf...
4,cardiovascular diseases,Spontaneous rupture of an aortic aneurysm into...


In [3]:
# ==============================================
# Split Medical Dataset into train/valid/test
# ==============================================
print("\n" + "=" * 60)
print("Split Medical Dataset into train/valid/test")
print("=" * 60 + "\n")

# Perform the split
train_size, valid_size, test_size = 0.7, 0.15, 0.15
train_df, valid_df, test_df = split_medical_dataset(data,train_size, valid_size, test_size)

# Display summary information
print(f"Training set: {train_df.shape[0]} rows × {train_df.shape[1]} columns")
display(train_df.head(3))

print(f"Validation set: {valid_df.shape[0]} rows × {valid_df.shape[1]} columns")
display(valid_df.head(3))

print(f"Test set: {test_df.shape[0]} rows × {test_df.shape[1]} columns")
display(test_df.head(3))


2025-10-07 14:01:39,164 - mednlpix.data.splitter - INFO - Split - Train: (1600, 2), Valid: (343, 2), Test: (343, 2)



Split Medical Dataset into train/valid/test

Training set: 1600 rows × 2 columns


,condition_label,medical_abstract
300,cardiovascular diseases,Recurrent intracaval renal cell carcinoma: the...
771,general pathological conditions,Anterior cervical discectomy and fusion. A ret...
1085,general pathological conditions,Stiffman syndrome: a rare paraneoplastic disor...


Validation set: 343 rows × 2 columns


,condition_label,medical_abstract
1270,neoplasms,Primary prevention of cancer. The case for com...
1264,general pathological conditions,Spontaneous rupture of a normal bladder. We ha...
1933,nervous system diseases,Orbital intramuscular schwannoma. In an 8-year...


Test set: 343 rows × 2 columns


,condition_label,medical_abstract
785,general pathological conditions,Urinary tract infections. Urinary tract infect...
436,digestive system diseases,Endoscopic appearance and significance of func...
1831,neoplasms,Flexible sigmoidoscopy as a screening procedur...


In [2]:
from mednlpix.features.preprocessing import MedicalTextPreprocessor

2025-10-07 14:40:05,342 - mednlpix.features.preprocessing.setup.nltk_setup - INFO - Downloading NLTK resource 'stopwords' to /Users/surelmanda/Data-Science-Projects/MedNLPix/src/mednlpix/features/resources/nltk_data...
[nltk_data] Downloading package stopwords to /Users/surelmanda/Data-
[nltk_data]     Science-Projects/MedNLPix/src/mednlpix/features/resour
[nltk_data]     ces/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
2025-10-07 14:40:05,763 - mednlpix.features.preprocessing.setup.nltk_setup - INFO - Downloading NLTK resource 'punkt' to /Users/surelmanda/Data-Science-Projects/MedNLPix/src/mednlpix/features/resources/nltk_data...
[nltk_data] Downloading package punkt to /Users/surelmanda/Data-
[nltk_data]     Science-Projects/MedNLPix/src/mednlpix/features/resour
[nltk_data]     ces/nltk_data...
[nltk_data] Error downloading 'punkt' from
[nltk_data]     <https://raw.githubusercontent.com/nltk/nltk_data/gh-
[nltk_data]     pages/packages/tokenizers/punkt.zip>:   [Errno 2

OSError: [Errno 28] No space left on device: '/Users/surelmanda/Data-Science-Projects/MedNLPix/src/mednlpix/logger/logs/mednlpix.features.preprocessing.text_cleaner.log'

In [9]:
import sys
from pathlib import Path

ROOT = Path(__file__).resolve().parents[3]
sys.path.append(str(ROOT))


NameError: name '__file__' is not defined

In [5]:
from mednlpix.features.preprocessing import MedicalTextPreprocessor

# Instantiate and fit on training data
preprocessor = MedicalTextPreprocessor()

train_clean = preprocessor.fit_transform(train_df)
valid_clean = preprocessor.transform(valid_df)
test_clean  = preprocessor.transform(test_df)

print(f"Train cleaned: {train_clean.shape}")
print(f"Valid cleaned: {valid_clean.shape}")
print(f"Test cleaned: {test_clean.shape}")

display(train_clean.head())


ModuleNotFoundError: No module named 'mednlpix.config'